In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
df=pd.read_csv("heart_clean_final.csv")

In [5]:
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,62,0,0,138,294.0,1,1,106,0,1.9,1,3,2,0
1,58,0,0,100,248.0,0,0,122,0,1.0,1,0,2,1
2,58,1,0,114,318.0,0,2,140,0,3.5,0,3,1,0
3,54,1,0,122,286.0,0,0,116,1,3.2,1,2,2,0
4,71,0,0,112,149.0,0,1,125,0,1.6,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,60,0,2,120,178.0,1,1,100,0,0.0,2,0,2,1
179,64,1,2,140,335.0,0,1,158,0,0.0,2,0,2,0
180,68,0,2,120,211.0,0,0,115,0,1.5,1,0,2,1
181,44,0,2,108,141.0,0,1,175,0,0.6,1,0,2,1


In [9]:
indep=df.drop("target",axis=1)
dep=df["target"]

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(indep, dep, test_size = 1/3, random_state = 0)

In [13]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train_= sc.fit_transform(X_train)
X_test_= sc.transform(X_test)

In [15]:
from sklearn.linear_model import LogisticRegression

In [97]:
from sklearn.model_selection import GridSearchCV
param_grid=[{'solver':['lbfgs','newton-cg','sag'],'penalty':['l2'],'C':[0.01,0.1,1,10,100],'max_iter':[1500]},
            {'solver':['liblinear'],'penalty':['l1','l2'],'C':[0.01,0.1,1,10,100]},
            {'solver':['saga'],'penalty':['elasticnet'],'l1_ratio':[0,0.5,1],'C':[0.01,0.1,1,10,100]}]
grid = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted')
grid.fit(X_train, y_train) 

Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid=[{'C': [0.01, 0.1, 1, 10, 100], 'max_iter': [1500],
                          'penalty': ['l2'],
                          'solver': ['lbfgs', 'newton-cg', 'sag']},
                         {'C': [0.01, 0.1, 1, 10, 100], 'penalty': ['l1', 'l2'],
                          'solver': ['liblinear']},
                         {'C': [0.01, 0.1, 1, 10, 100], 'l1_ratio': [0, 0.5, 1],
                          'penalty': ['elasticnet'], 'solver': ['saga']}],
             scoring='f1_weighted', verbose=3)

In [99]:
re=grid.cv_results_
grid_predictions=grid.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions) 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [101]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 10, 'max_iter': 1500, 'penalty': 'l2', 'solver': 'lbfgs'}: 0.8688524590163934


In [103]:
print("confusion matrix:\n",cm)

confusion matrix:
 [[ 8  4]
 [ 4 45]]


In [105]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.67      0.67      0.67        12
           1       0.92      0.92      0.92        49

    accuracy                           0.87        61
   macro avg       0.79      0.79      0.79        61
weighted avg       0.87      0.87      0.87        61



In [107]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(X_test)[:,1])

0.8690476190476191

In [109]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_max_iter,param_penalty,param_solver,param_l1_ratio,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.015110,0.002119,0.003999,0.001095,0.01,1500.0,l2,lbfgs,NaN,"{'C': 0.01, 'max_iter': 1500, 'penalty': 'l2',...",0.674312,0.785031,0.750000,0.500000,0.675000,0.676869,0.098326,25
1,0.008713,0.001469,0.002802,0.000400,0.01,1500.0,l2,newton-cg,NaN,"{'C': 0.01, 'max_iter': 1500, 'penalty': 'l2',...",0.674312,0.785031,0.750000,0.500000,0.675000,0.676869,0.098326,25
2,0.025264,0.003447,0.003705,0.001943,0.01,1500.0,l2,sag,NaN,"{'C': 0.01, 'max_iter': 1500, 'penalty': 'l2',...",0.674312,0.785031,0.750000,0.648148,0.675000,0.706498,0.051947,20
3,0.042820,0.011159,0.003470,0.000776,0.10,1500.0,l2,lbfgs,NaN,"{'C': 0.1, 'max_iter': 1500, 'penalty': 'l2', ...",0.821754,0.833016,0.795455,0.648148,0.808772,0.781429,0.067817,13
4,0.009910,0.001963,0.003102,0.000201,0.10,1500.0,l2,newton-cg,NaN,"{'C': 0.1, 'max_iter': 1500, 'penalty': 'l2', ...",0.821754,0.833016,0.795455,0.648148,0.808772,0.781429,0.067817,13
5,0.026563,0.002888,0.002971,0.000524,0.10,1500.0,l2,sag,NaN,"{'C': 0.1, 'max_iter': 1500, 'penalty': 'l2', ...",0.767366,0.785031,0.795455,0.589066,0.675000,0.722384,0.079128,18
6,0.069277,0.008765,0.002904,0.000493,1.00,1500.0,l2,lbfgs,NaN,"{'C': 1, 'max_iter': 1500, 'penalty': 'l2', 's...",0.785031,0.833016,0.871978,0.701282,0.808772,0.800016,0.057126,5
7,0.013952,0.003538,0.003001,0.000631,1.00,1500.0,l2,newton-cg,NaN,"{'C': 1, 'max_iter': 1500, 'penalty': 'l2', 's...",0.785031,0.833016,0.871978,0.701282,0.808772,0.800016,0.057126,5
8,0.026293,0.002765,0.002906,0.000859,1.00,1500.0,l2,sag,NaN,"{'C': 1, 'max_iter': 1500, 'penalty': 'l2', 's...",0.767366,0.785031,0.795455,0.589066,0.675000,0.722384,0.079128,18
9,0.085594,0.009692,0.002906,0.000804,10.00,1500.0,l2,lbfgs,NaN,"{'C': 10, 'max_iter': 1500, 'penalty': 'l2', '...",0.785031,0.877714,0.871978,0.701282,0.771704,0.801542,0.066300,1
